In [1]:
import sqlite3
import pandas as pd

### Создание базы данных о полевках из ЗИНа

Для начала создадим базу с характеристиками митохондриальных геномов полевочьих

In [2]:
df = pd.read_csv('mt_stats.tsv', sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Tribe      40 non-null     object 
 1   Genus      40 non-null     object 
 2   Species    40 non-null     object 
 3   tissue     40 non-null     object 
 4   collector  23 non-null     object 
 5   year       18 non-null     float64
 6   T          40 non-null     float64
 7   A          40 non-null     float64
 8   C          40 non-null     float64
 9   G          40 non-null     float64
 10  GC count   40 non-null     float64
 11  GC skew    40 non-null     float64
dtypes: float64(7), object(5)
memory usage: 3.9+ KB


Создадим базу и таблицу с характеристиками

In [3]:
with sqlite3.connect('voles.db') as conn:
    conn.execute('PRAGMA FOREIGN_KEYS=ON')
    c = conn.cursor()
    c.execute('PRAGMA FOREIGN_KEYS=ON')
    c.execute('''
                CREATE TABLE mt_stats (
                [Tribe] text,
                [Genus] text,
                [Species] text PRIMARY KEY,
                [tissue] text,
                [collector] text,
                [year] integer,
                [T] integer,
                [A] integer,
                [C] integer,
                [G] integer,
                [GC count] float,
                [GC skew] float)
             ''')
    df.to_sql('mt_stats', conn, if_exists='append', index=False)

/home/asha/ИБ/python/SQL/venv/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


Заполним таблицу данными

In [4]:
# Проверяем, что записи добавились и тестируем запрос

with sqlite3.connect('voles.db') as conn:
    c = conn.cursor()
    c.execute('''
              SELECT T, G FROM mt_stats
              WHERE Genus='Microtus'
              ''')
    print(c.fetchall())

[(27.9, 13.4), (28.3, 13.6), (27.4, 14.4), (28.3, 12.8), (28.5, 13.4), (30.8, 13), (27.7, 13.6), (27.8, 13.1), (28.4, 13.1), (28.6, 13.3), (27.9, 13.3)]


### Добавим еще одну таблицу в базу данных 
Создадим таблицу для транскриптомных данных, для нее свяжем виды с предыдущей таблицей по ключу Species

In [5]:
df = pd.read_csv('trinity_stats.tsv', sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Species                    29 non-null     object 
 1   № base / server            19 non-null     object 
 2   Total trinity 'genes'      27 non-null     float64
 3   Total trinity transcripts  27 non-null     float64
 4   Percent GC                 27 non-null     float64
 5   N10                        27 non-null     float64
 6   N20                        27 non-null     float64
 7   N30                        27 non-null     float64
 8   N40                        27 non-null     float64
 9   N50                        27 non-null     float64
 10  Median contig length       27 non-null     float64
 11  Average contig             27 non-null     float64
 12  Total assembled bases      27 non-null     float64
 13  Source                     29 non-null     object 
d

In [6]:
with sqlite3.connect('voles.db') as conn:
    conn.execute('PRAGMA FOREIGN_KEYS=ON')
    c = conn.cursor()
    c.execute('''CREATE TABLE trinity_stats(
                 [id] integer PRIMARY KEY,
                 [Species] text,
                 [№ base / server] text,
                 [Total trinity 'genes'] integer,
                 [Total trinity transcripts] integer,
                 [Percent GC] integer,
                 [N10] integer,
                 [N20] integer,
                 [N30] integer,
                 [N40] integer,
                 [N50] integer,
                 [Median contig length] integer,
                 [Average contig] float,
                 [Total assembled bases] integer,
                 [Source] text,
                 FOREIGN KEY (Species)
                 REFERENCES mt_stats (Species)
                    ON DELETE CASCADE)''')
conn.commit()

In [7]:
with sqlite3.connect('voles.db') as conn:
    c.execute('PRAGMA FOREIGN_KEYS=ON')
    df.to_sql('trinity_stats', conn, if_exists='append', index=False)
    conn.commit()

In [8]:
# Проверяем, что записи добавились и тестируем запрос

with sqlite3.connect('voles.db') as conn:
    c = conn.cursor()
    c.execute('''
              SELECT Species, N50 FROM trinity_stats
              WHERE N50>=2300
              ''')
    print(c.fetchall())

[('L. brandtii', 2573), ('L. mandarinus', 2552), ('M. pennsylvanicus', 2388), ('A. amphibius', 2343), ('E. lutescens', 2569), ('C. glareolus', 2619), ('L. gregalis', 3043), ('L. raddei', 3571), ('T. subterraneus', 2508), ('A. lemminus', 2679), ('L. sibiricus', 2580), ('M. schisticolor', 2579), ('P. schaposchnikowi', 2890), ('C. nivalis', 3219), ('Clethrionomys sp.', 2478), ('A. tuvinicus', 2444), ('A. macrotis', 2963), ('L. lagurus', 2880)]


#### Изменим значения в базе в колонке Source, вместо "наш" подставим "ZIN"


In [9]:
with sqlite3.connect('voles.db') as conn:
    c = conn.cursor()
    c.execute('''
                UPDATE trinity_stats
                SET Source = 'ZIN'
                WHERE Source = 'наш'; 
              ''')

In [10]:
# Проверка

with sqlite3.connect('voles.db') as conn:
    c = conn.cursor()
    c.execute('''
              SELECT Species FROM trinity_stats
              WHERE Source = 'genome'
              ''')
    print(c.fetchall())

[]


#### Удалим те записи, где источник данных - genome

In [11]:
with sqlite3.connect('voles.db') as conn:
    c = conn.cursor()
    c.execute('''
                DELETE FROM trinity_stats WHERE Source='genome';
              ''')

In [12]:
# Проверка

with sqlite3.connect('voles.db') as conn:
    c = conn.cursor()
    c.execute('''
              SELECT Species FROM trinity_stats
              WHERE Source = 'genome'
              ''')
    print(c.fetchall())

[]


### CASCADE delete
Попробуем удалить запись из таблицы с данными о сборке представителя вида C. nivalis. Данные о его митогеноме должны быть также удалены

In [13]:
# Тестируем запрос

with sqlite3.connect('voles.db') as conn:
    c = conn.cursor()
    c.execute('''
              SELECT * FROM mt_stats
              WHERE Species='C. nivalis'
              ''')
    print(c.fetchall())
    

[('Arvicolini', 'Chionomys', 'C. nivalis', '2737', 'Стекольников АА', 2009, 28, 31, 27.7, 13.3, 35.6735, -0.3176)]


In [14]:
# Тестируем запрос

with sqlite3.connect('voles.db') as conn:
    c = conn.cursor()
    c.execute('''
              SELECT * FROM trinity_stats
              WHERE Species='C. nivalis'
              ''')
    print(c.fetchall())

[(19, 'C. nivalis', '5440, 5441', 46957, 71753, 49.78, 7523, 5764, 4673, 3904, 3219, 775, 1592.66, 114277997, 'ZIN')]


In [15]:
with sqlite3.connect('voles.db') as conn:
    conn.execute('PRAGMA FOREIGN_KEYS=ON')
    c = conn.cursor()
    c.execute('''
                DELETE FROM mt_stats WHERE Species='C. nivalis';
              ''')

In [16]:
# Тестируем запрос

with sqlite3.connect('voles.db') as conn:
    c = conn.cursor()
    c.execute('''
              SELECT * FROM trinity_stats
              WHERE Species='C. nivalis'
              ''')
    print(c.fetchall())

[]


In [17]:
# Тестируем запрос

with sqlite3.connect('voles.db') as conn:
    c = conn.cursor()
    c.execute('''
              SELECT * FROM mt_stats
              WHERE Species='C. nivalis'
              ''')
    print(c.fetchall())

[]
